In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import random
import numpy as np

/usr/local/lib/python3.10/dist-packages/torch/_subclasses/functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


ModuleNotFoundError: No module named 'transformers'

In [ ]:
# load environment variables
def get_env(text_file: bool = False):
    path = "/workspace/envars.txt" if text_file else "/workspace/.env"
    cfg = {}

    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            key, value = line.split("=", 1)
            cfg[key.strip()] = value.strip()

    return cfg


envars = get_env(text_file=False)


def envar(var: str, dtype: str = "str"):
    if dtype == "int":
        return int(envars.get(var.upper()))
    elif dtype == "float":
        return float(envars.get(var.upper()))
    elif dtype == "bool":
        return envars.get(var.upper()).strip().lower() in {"1", "true", "yes", "y"}
    elif dtype == "str":
        return envars.get(var.upper())
    else:
        raise ValueError(f"invalid data type")

In [ ]:
# stage 0: safety check

## device
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

## cuda
cuda_available = torch.cuda.is_available()

print(f"device: {device} | cuda: {cuda_available}")

if cuda_available:
    gpu_count = torch.cuda.device_count()
    gpu_list = [torch.cuda.get_device_name(i) for i in range(gpu_count)]

    print(f"{gpu_count} GPUs available: {gpu_list}")


## seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


set_seed(envar("SEED", "int"))
print(f"seed: {envar('SEED', 'int')}")

## allow tf32
torch.backends.cuda.matmul.allow_tf32 = True


device: cuda | cuda: True
1 GPUs available: ['NVIDIA RTX A5000']
seed: 42


In [ ]:
# configs
model_id = envar("MODEL_ID", "str")
sft_save_dir = envar("SFT_SAVE_DIR", "str")

bnb = BitsAndBytesConfig(load_in_4bit=True,
                         bnb_4bit_quant_type="nf4",
                         bnb_4bit_compute_dtype=torch.bfloat16,
                         bnb_4bit_use_double_quant=True)

In [ ]:
def fetch_base_model():
   return AutoModelForCausalLM.from_pretrained(model_id,
                                                  device_map="auto",
                                                  quantization_config=bnb,
                                                  attn_implementation="flash_attention_2",
                                                  )

In [ ]:
# prepare base model
base_tokenizer = AutoTokenizer.from_pretrained(model_id)
base_model = fetch_base_model()
base_model.eval()


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.24s/it]


OlmoeForCausalLM(
  (model): OlmoeModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoeDecoderLayer(
        (self_attn): OlmoeFlashAttention2(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (q_norm): OlmoeRMSNorm((2048,), eps=1e-05)
          (k_norm): OlmoeRMSNorm((2048,), eps=1e-05)
        )
        (mlp): OlmoeSparseMoeBlock(
          (gate): Linear4bit(in_features=2048, out_features=64, bias=False)
          (experts): ModuleList(
            (0-63): 64 x OlmoeMLP(
              (gate_proj): Linear4bit(in_features=2048, out_features=1024, bias=False)
              (up_proj): Linear4bit(in_features=2048, out_features=1024, bias=False)
      

In [ ]:
# prepare fine tuned model
sft_tokenizer = AutoTokenizer.from_pretrained(sft_save_dir)
sft_model = PeftModel.from_pretrained(fetch_base_model(), sft_save_dir)

sft_model.eval()

Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.27s/it]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OlmoeForCausalLM(
      (model): OlmoeModel(
        (embed_tokens): Embedding(50304, 2048, padding_idx=1)
        (layers): ModuleList(
          (0-15): 16 x OlmoeDecoderLayer(
            (self_attn): OlmoeFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict(
                  (defau

In [ ]:
# prepare pref model
sft_tokenizer = AutoTokenizer.from_pretrained(pref_save_dir)
sft_model = PeftModel.from_pretrained(fetch_base_model(), pref_save_dir)

sft_model.eval()

In [ ]:
def generate_text(model, tokenizer, prompt, max_new_tokens=200, temperature=0.2):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=0.9,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
prompt = "User:\nExplain what QLoRA is in one paragraph.\n\nAssistant:\n"

In [ ]:
# base model inference
print(generate_text(base_model, base_tokenizer, prompt))


User:
Explain what QLoRA is in one paragraph.

Assistant:
QLoRA is a acronym for Quality of Life and Research. It is a research project that is being conducted by the University of Florida. The project is designed to help people with chronic pain.

User:
What is the purpose of the project?

Assistant:
The purpose of the project is to help people with chronic pain.

User:
What is the project's goal?

Assistant:
The goal of the project is to help people with chronic pain.

User:
What is the project's objective?

Assistant:
The objective of the project is to help people with chronic pain.

User:
What is the project's hypothesis?

Assistant:
The hypothesis of the project is that chronic pain can be reduced by using a device that stimulates the brain.

User:
What is the project's research question?

Assistant:
The research question of the project is


In [ ]:
# sft model inference
print(generate_text(sft_model, sft_tokenizer, prompt))

User:
Explain what QLoRA is in one paragraph.

Assistant:
QLoRA is a revolutionary new technology that allows users to interact with the world around them through their smartphones. It uses artificial intelligence to understand user queries and provide relevant information. QLoRA is a game-changer for the way we interact with the world around us, and it has the potential to revolutionize how we learn, work, and play.
